# Phase 4 Project Notebook
- Author: Jonathan Holt
- Data Science Flex

## Business Problem
- What are the top 5 best zip codes for us to invest in?

## Questions to Answer

1. What is the average (mean) home price for zip codes?
    a. also by: State, Metro Region, etc.
2. Which zip codes performed above expectations?
3. Do the top performing zip codes have any identifiers?


## What Models & Metrics I plan on using

## Helper Functions
- Functions provided by Flatiron

In [1]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

In [2]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [3]:
#function for displaying money in millions.
def display_millions(x, pos):
    return '${:1.1f}M'.format(x*1e-6)

## Loading Data

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.ticker as mtick
import seaborn as sns
from statsmodels.tsa.stattools import adfuller

plt.style.use('seaborn')
pd.set_option('display.max_rows', 1500) #change the amount of rows displayed

pd.options.display.float_format = '{:,.2f}'.format 

In [5]:
df = pd.read_csv("zillow_data.csv")

In [6]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,"334,200.00","335,400.00","336,500.00",...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,"235,700.00","236,900.00","236,700.00",...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,"210,400.00","212,200.00","212,200.00",...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,"498,100.00","500,900.00","503,100.00",...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,"77,300.00","77,300.00","77,300.00",...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


##  Fixing RegionName
A google search shows that RegionName is the ZipCode for each Region. However, upon sorting, I discovered that any ZipCode beginning with a 0 was ignoring it and displaying as a 4 digit number. I will use the .str().zfill() method to ensure that all RegionNames are displaying the as 5 digits.

In [7]:
df['RegionName'] = df['RegionName'].astype(str).str.zfill(5)

In [8]:
# Change to ZipCode?

## Checking for Null Values

In [9]:
df.isnull().sum()

RegionID         0
RegionName       0
City             0
State            0
Metro         1043
CountyName       0
SizeRank         0
1996-04       1039
1996-05       1039
1996-06       1039
1996-07       1039
1996-08       1039
1996-09       1039
1996-10       1039
1996-11       1039
1996-12       1039
1997-01       1039
1997-02       1039
1997-03       1039
1997-04       1039
1997-05       1039
1997-06       1039
1997-07       1038
1997-08       1038
1997-09       1038
1997-10       1038
1997-11       1038
1997-12       1038
1998-01       1036
1998-02       1036
1998-03       1036
1998-04       1036
1998-05       1036
1998-06       1036
1998-07       1036
1998-08       1036
1998-09       1036
1998-10       1036
1998-11       1036
1998-12       1036
1999-01       1036
1999-02       1036
1999-03       1036
1999-04       1036
1999-05       1036
1999-06       1036
1999-07       1036
1999-08       1036
1999-09       1036
1999-10       1036
1999-11       1036
1999-12       1036
2000-01     

### Analysis
There are many Null Values. 
- For categorical data, 7% of Metro are null (1043 of 14,723).
- Dates from 1996 - mid 2003, also have 7% null values.
- Then it starts to get better. 6% null and decreasing.

What is my decision on null values?
- keep?
- delete?
- change (mean)?

## Dealing with Nulls

DEAL WITH NULLS HERE!

### Metro

In [10]:
df['Metro'].value_counts()

New York                              779
Los Angeles-Long Beach-Anaheim        347
Chicago                               325
Philadelphia                          281
Washington                            249
Boston                                246
Dallas-Fort Worth                     217
Minneapolis-St Paul                   201
Houston                               187
Pittsburgh                            177
Miami-Fort Lauderdale                 162
Portland                              161
Detroit                               153
Atlanta                               152
Seattle                               141
St. Louis                             140
San Francisco                         134
Kansas City                           127
Phoenix                               126
Baltimore                             122
Tampa                                 118
Riverside                             116
Cincinnati                            109
Denver                            

In [11]:
df.sort_values('Metro').tail()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
14684,70215,28775,Scaly Mountain,NC,NaN,Macon,14685,"71,800.00","71,900.00","72,000.00",...,215900,215600,217400,219700,218900,215100,211400,213000,216500,216700
14694,99114,97149,Neskowin,OR,NaN,Tillamook,14695,"187,900.00","188,900.00","189,700.00",...,394800,399000,402900,408800,418900,428100,433000,431600,421600,413400
14710,59210,03812,Bartlett,NH,NaN,Carroll,14711,"80,900.00","80,800.00","80,800.00",...,215500,217000,219200,221700,223600,224800,226000,226900,227700,228000
14717,62697,12720,Bethel,NY,NaN,Sullivan,14718,"62,500.00","62,600.00","62,700.00",...,122200,122700,122300,122000,122200,122800,123200,123200,120700,117700
14721,93733,81225,Mount Crested Butte,CO,NaN,Gunnison,14722,"191,100.00","192,400.00","193,700.00",...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


### Analysis:
- I think it's okay to leave the NaN values for now as these records appear to be in rural areas where there wouldn't be close enough to a city to be considered part of the Metro area. 
- I only plan on using Metro area to look at cities anyway, so there shouldn't be a penalty or unneccessary exlusion on any of the records lacking Metro values

# Feature Engineering

- add: Region Mean, Region Growth, etc
- THE POINT IS TO IDENTIFY WHICH ZIPS ARE THE BEST PERFORMING SO I CAN NARROW DOWN THE DATA TO CAPTURE!!!!

In [12]:
# total_profit = table9.apply(lambda x: x['worldwide_gross'] - x['production_budget'], axis=1)
# table9['total_profit'] = total_profit

## Recent Data
- only the data from 2013-2018

In [13]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,"334,200.00","335,400.00","336,500.00",...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,"235,700.00","236,900.00","236,700.00",...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,"210,400.00","212,200.00","212,200.00",...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,"498,100.00","500,900.00","503,100.00",...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,"77,300.00","77,300.00","77,300.00",...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [14]:
recent_data = df.iloc[:, -64:]
recent_data.head()

,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,"802,300.00","806,100.00","810,900.00","817,400.00","826,800.00","837,900.00","848,100.00","853,800.00","856,700.00","856,600.00",...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,"205,900.00","206,900.00","208,500.00","209,800.00","211,300.00","214,000.00","217,200.00","220,600.00","223,800.00","226,500.00",...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,"256,900.00","256,800.00","256,700.00","257,100.00","258,300.00","260,700.00","263,900.00","267,000.00","269,200.00","271,000.00",...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,"1,006,300.00","1,013,700.00","1,024,800.00","1,038,300.00","1,053,900.00","1,070,600.00","1,089,900.00","1,108,100.00","1,123,700.00","1,135,100.00",...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,"113,000.00","113,300.00","113,600.00","113,500.00","113,300.00","113,000.00","113,000.00","112,900.00","112,800.00","112,500.00",...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


In [15]:
recent_data.apply(lambda x: round(x, 2))

,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,"802,300.00","806,100.00","810,900.00","817,400.00","826,800.00","837,900.00","848,100.00","853,800.00","856,700.00","856,600.00",...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,"205,900.00","206,900.00","208,500.00","209,800.00","211,300.00","214,000.00","217,200.00","220,600.00","223,800.00","226,500.00",...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,"256,900.00","256,800.00","256,700.00","257,100.00","258,300.00","260,700.00","263,900.00","267,000.00","269,200.00","271,000.00",...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,"1,006,300.00","1,013,700.00","1,024,800.00","1,038,300.00","1,053,900.00","1,070,600.00","1,089,900.00","1,108,100.00","1,123,700.00","1,135,100.00",...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,"113,000.00","113,300.00","113,600.00","113,500.00","113,300.00","113,000.00","113,000.00","112,900.00","112,800.00","112,500.00",...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,"171,800.00","173,800.00","176,800.00","179,700.00","181,500.00","182,300.00","182,200.00","182,800.00","183,200.00","183,600.00",...,216800,217700,218600,218500,218100,216400,213100,209800,209200,209300
14719,"173,500.00","173,500.00","172,200.00","171,000.00","172,100.00","174,000.00","175,800.00","177,700.00","179,100.00","179,200.00",...,202100,208400,212200,215200,214300,213100,213700,218300,222700,225800
14720,"98,800.00","99,100.00","99,000.00","98,700.00","99,000.00","100,300.00","101,300.00","101,700.00","102,700.00","104,100.00",...,121800,122800,124600,126700,128800,130600,131700,132500,133000,133400
14721,"516,200.00","520,300.00","526,900.00","531,200.00","533,700.00","535,700.00","538,400.00","538,300.00","540,200.00","537,700.00",...,662800,671200,682400,695600,695500,694700,706400,705300,681500,664400


In [16]:
recent_data['mean'] = recent_data.apply(lambda x: x.mean(), axis=1)

In [17]:
recent_data['delta'] = recent_data.apply(lambda x: x['2018-04'] - x['2013-01'], axis=1)

In [18]:
recent_data

,2013-01,2013-02,2013-03,2013-04,2013-05,2013-06,2013-07,2013-08,2013-09,2013-10,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,mean,delta
0,"802,300.00","806,100.00","810,900.00","817,400.00","826,800.00","837,900.00","848,100.00","853,800.00","856,700.00","856,600.00",...,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600,"929,381.25","228,300.00"
1,"205,900.00","206,900.00","208,500.00","209,800.00","211,300.00","214,000.00","217,200.00","220,600.00","223,800.00","226,500.00",...,312500,314100,315000,316600,318100,319600,321100,321800,"267,568.75","115,900.00"
2,"256,900.00","256,800.00","256,700.00","257,100.00","258,300.00","260,700.00","263,900.00","267,000.00","269,200.00","271,000.00",...,320200,320400,320800,321200,321200,323000,326900,329900,"304,398.44","73,000.00"
3,"1,006,300.00","1,013,700.00","1,024,800.00","1,038,300.00","1,053,900.00","1,070,600.00","1,089,900.00","1,108,100.00","1,123,700.00","1,135,100.00",...,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000,"1,201,653.12","300,700.00"
4,"113,000.00","113,300.00","113,600.00","113,500.00","113,300.00","113,000.00","113,000.00","112,900.00","112,800.00","112,500.00",...,120000,120300,120300,120300,120300,120500,121000,121500,"115,440.62","8,500.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14718,"171,800.00","173,800.00","176,800.00","179,700.00","181,500.00","182,300.00","182,200.00","182,800.00","183,200.00","183,600.00",...,218600,218500,218100,216400,213100,209800,209200,209300,"193,946.88","37,500.00"
14719,"173,500.00","173,500.00","172,200.00","171,000.00","172,100.00","174,000.00","175,800.00","177,700.00","179,100.00","179,200.00",...,212200,215200,214300,213100,213700,218300,222700,225800,"188,935.94","52,300.00"
14720,"98,800.00","99,100.00","99,000.00","98,700.00","99,000.00","100,300.00","101,300.00","101,700.00","102,700.00","104,100.00",...,124600,126700,128800,130600,131700,132500,133000,133400,"111,531.25","34,600.00"
14721,"516,200.00","520,300.00","526,900.00","531,200.00","533,700.00","535,700.00","538,400.00","538,300.00","540,200.00","537,700.00",...,682400,695600,695500,694700,706400,705300,681500,664400,"570,803.12","148,200.00"


## attempting merge

In [19]:
#recent_stats = recent_data.iloc[:, -3:]

In [20]:
new_df = df.iloc[:, :7]
new_df = new_df.reset_index()

In [21]:
#new_df.drop([:, -1])

In [22]:
recent_data = recent_data.reset_index()

In [23]:
#cs_churn_df = cs_churn_df.merge(variable_1, on='#_of_calls')
new_df = new_df.merge(recent_data, on='index')

In [24]:
new_df.head()

,index,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2013-01,2013-02,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,mean,delta
0,0,84654,60657,Chicago,IL,Chicago,Cook,1,"802,300.00","806,100.00",...,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600,"929,381.25","228,300.00"
1,1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,"205,900.00","206,900.00",...,312500,314100,315000,316600,318100,319600,321100,321800,"267,568.75","115,900.00"
2,2,91982,77494,Katy,TX,Houston,Harris,3,"256,900.00","256,800.00",...,320200,320400,320800,321200,321200,323000,326900,329900,"304,398.44","73,000.00"
3,3,84616,60614,Chicago,IL,Chicago,Cook,4,"1,006,300.00","1,013,700.00",...,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000,"1,201,653.12","300,700.00"
4,4,93144,79936,El Paso,TX,El Paso,El Paso,5,"113,000.00","113,300.00",...,120000,120300,120300,120300,120300,120500,121000,121500,"115,440.62","8,500.00"


In [25]:
df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04
0,84654,60657,Chicago,IL,Chicago,Cook,1,"334,200.00","335,400.00","336,500.00",...,1005500,1007500,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,"235,700.00","236,900.00","236,700.00",...,308000,310000,312500,314100,315000,316600,318100,319600,321100,321800
2,91982,77494,Katy,TX,Houston,Harris,3,"210,400.00","212,200.00","212,200.00",...,321000,320600,320200,320400,320800,321200,321200,323000,326900,329900
3,84616,60614,Chicago,IL,Chicago,Cook,4,"498,100.00","500,900.00","503,100.00",...,1289800,1287700,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000
4,93144,79936,El Paso,TX,El Paso,El Paso,5,"77,300.00","77,300.00","77,300.00",...,119100,119400,120000,120300,120300,120300,120300,120500,121000,121500


### ASSERT NEEDED: new_df has accurate data, but before turning in this project I should include ASSERT to prove that it is.

# Analysis of New DF w/Metrics

In [26]:
new_df.drop(columns=['index'], inplace=True)

In [27]:
new_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2013-01,2013-02,2013-03,...,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,mean,delta
0,84654,60657,Chicago,IL,Chicago,Cook,1,"802,300.00","806,100.00","810,900.00",...,1007800,1009600,1013300,1018700,1024400,1030700,1033800,1030600,"929,381.25","228,300.00"
1,90668,75070,McKinney,TX,Dallas-Fort Worth,Collin,2,"205,900.00","206,900.00","208,500.00",...,312500,314100,315000,316600,318100,319600,321100,321800,"267,568.75","115,900.00"
2,91982,77494,Katy,TX,Houston,Harris,3,"256,900.00","256,800.00","256,700.00",...,320200,320400,320800,321200,321200,323000,326900,329900,"304,398.44","73,000.00"
3,84616,60614,Chicago,IL,Chicago,Cook,4,"1,006,300.00","1,013,700.00","1,024,800.00",...,1287400,1291500,1296600,1299000,1302700,1306400,1308500,1307000,"1,201,653.12","300,700.00"
4,93144,79936,El Paso,TX,El Paso,El Paso,5,"113,000.00","113,300.00","113,600.00",...,120000,120300,120300,120300,120300,120500,121000,121500,"115,440.62","8,500.00"


In [29]:
new_df = new_df.sort_values('delta', ascending=False)

In [33]:
new_df['delta_rank'] = range(len(df))
new_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2013-01,2013-02,2013-03,...,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,mean,delta,delta_rank
272,61635,10021,New York,NY,New York,New York,273,"11,151,000.00","11,168,300.00","11,191,400.00",...,18569400,18428800,18307100,18365900,18530400,18337700,17894900,"16,281,670.31","6,743,900.00",0
20,61625,10011,New York,NY,New York,New York,21,"7,886,000.00","7,814,000.00","7,849,900.00",...,12050100,12016300,11946500,11978100,11849300,11563000,11478300,"10,555,268.75","3,592,300.00",1
10237,97518,94027,Atherton,CA,San Francisco,San Mateo,10238,"3,753,100.00","3,832,300.00","3,913,100.00",...,6315400,6429100,6581800,6705000,6764600,6788400,6796500,"5,228,923.44","3,043,400.00",2
4816,96086,90210,Beverly Hills,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,4817,"3,181,200.00","3,232,900.00","3,301,000.00",...,5503000,5574100,5661000,5738200,5816100,5907800,5956700,"4,501,590.62","2,775,500.00",3
5738,97691,94301,Palo Alto,CA,San Jose,Santa Clara,5739,"2,536,400.00","2,577,200.00","2,612,500.00",...,4401100,4481700,4544700,4595100,4668700,4762600,4818400,"3,590,104.69","2,282,000.00",4


In [34]:
new_df = new_df.sort_values('mean', ascending=False)
new_df['mean_rank'] = range(len(df))
new_df.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2013-01,2013-02,2013-03,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,mean,delta,delta_rank,mean_rank
272,61635,10021,New York,NY,New York,New York,273,"11,151,000.00","11,168,300.00","11,191,400.00",...,18428800,18307100,18365900,18530400,18337700,17894900,"16,281,670.31","6,743,900.00",0,0
20,61625,10011,New York,NY,New York,New York,21,"7,886,000.00","7,814,000.00","7,849,900.00",...,12016300,11946500,11978100,11849300,11563000,11478300,"10,555,268.75","3,592,300.00",1,1
508,61628,10014,New York,NY,New York,New York,509,"7,177,700.00","7,143,400.00","7,247,200.00",...,9515800,9492800,9524900,9373400,9119600,8959300,"9,151,843.75","1,781,600.00",9,2
21,61703,10128,New York,NY,New York,New York,22,"5,378,000.00","5,411,800.00","5,480,100.00",...,7371400,7342700,7353300,7350300,7363000,7386600,"6,953,489.06","2,008,600.00",6,3
10237,97518,94027,Atherton,CA,San Francisco,San Mateo,10238,"3,753,100.00","3,832,300.00","3,913,100.00",...,6429100,6581800,6705000,6764600,6788400,6796500,"5,228,923.44","3,043,400.00",2,4


In [36]:
new_df.sort_values("delta_rank").head(1000)

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,2013-01,2013-02,2013-03,...,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,mean,delta,delta_rank,mean_rank
272,61635,10021,New York,NY,New York,New York,273,"11,151,000.00","11,168,300.00","11,191,400.00",...,18428800,18307100,18365900,18530400,18337700,17894900,"16,281,670.31","6,743,900.00",0,0
20,61625,10011,New York,NY,New York,New York,21,"7,886,000.00","7,814,000.00","7,849,900.00",...,12016300,11946500,11978100,11849300,11563000,11478300,"10,555,268.75","3,592,300.00",1,1
10237,97518,94027,Atherton,CA,San Francisco,San Mateo,10238,"3,753,100.00","3,832,300.00","3,913,100.00",...,6429100,6581800,6705000,6764600,6788400,6796500,"5,228,923.44","3,043,400.00",2,4
4816,96086,90210,Beverly Hills,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,4817,"3,181,200.00","3,232,900.00","3,301,000.00",...,5574100,5661000,5738200,5816100,5907800,5956700,"4,501,590.62","2,775,500.00",3,5
5738,97691,94301,Palo Alto,CA,San Jose,Santa Clara,5739,"2,536,400.00","2,577,200.00","2,612,500.00",...,4481700,4544700,4595100,4668700,4762600,4818400,"3,590,104.69","2,282,000.00",4,10
5843,97513,94022,Los Altos,CA,San Jose,Santa Clara,5844,"2,332,000.00","2,367,300.00","2,400,100.00",...,4045200,4118000,4170500,4235700,4321900,4372600,"3,244,868.75","2,040,600.00",5,13
21,61703,10128,New York,NY,New York,New York,22,"5,378,000.00","5,411,800.00","5,480,100.00",...,7371400,7342700,7353300,7350300,7363000,7386600,"6,953,489.06","2,008,600.00",6,3
4723,72636,33480,Palm Beach,FL,Miami-Fort Lauderdale,Palm Beach,4724,"2,795,900.00","2,825,800.00","2,860,700.00",...,4588200,4609000,4626500,4661200,4692300,4686200,"3,805,860.94","1,890,300.00",7,8
1534,62028,11217,New York,NY,New York,Kings,1535,"1,836,200.00","1,844,300.00","1,854,300.00",...,3407000,3404100,3413800,3482900,3588100,3656000,"2,819,290.62","1,819,800.00",8,19
508,61628,10014,New York,NY,New York,New York,509,"7,177,700.00","7,143,400.00","7,247,200.00",...,9515800,9492800,9524900,9373400,9119600,8959300,"9,151,843.75","1,781,600.00",9,2


# NEXT FEATURE = ADD DELTA AS A PERCENTAGE OF VALUE (IE, HOME VALUES INCREASED X%)

## Transforming into Date-Time
- columns cannot be turned to Date time without changing the data itself. I need to get the dates to become the index of my df and somehow keep all of the data within.

In [ ]:
df.head()

In [ ]:
data_df = df.iloc[:, 7:]
data_df

In [ ]:
data_df

In [ ]:
data_df.columns = pd.to_datetime(data_df.columns, format='%Y-%m')
data_df.head()

In [ ]:
data_df.iloc[:, 0:9].info()

In [ ]:
#get_datetimes(data)

In [ ]:
df.iloc[:, :7].head()

In [ ]:
df_1996 = df.iloc[:, :16]
df_1996

In [ ]:
df_1996.info()

In [ ]:
#this works!
df_1996['1996-04'] = pd.to_datetime(df_1996['1996-04'])

In [ ]:
#pd.to_datetime(df_1996[:, 7:])

In [ ]:
df_1996.info()

In [ ]:
df_1996

In [ ]:
df_1996.head()

In [ ]:
#df_1996.drop(columns={'RegionID', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'}, inplace=True)
df_1996.drop(columns={'RegionID', 'RegionName', 'City','Metro', 'CountyName', 'SizeRank'}, inplace=True)

In [ ]:
jon_df = df_1996
jon_df.head()

In [ ]:
jon_df_2 = jon_df.iloc[:, 1:]

In [ ]:
jon_df_2

In [ ]:
pd.melt(df_1996).tail(1000)

## Month Values

In [ ]:
df.sort_values('1996-04').tail()

In [ ]:
df.sort_values('1996-04').tail()

## Grouping by Zip (Region Name)

In [ ]:
zip_df = df.groupby('RegionName').mean()

In [ ]:
#zip_df = zip_df.reset_index()

In [ ]:
melt_data(zip_df)

### Melt

In [ ]:
pd.melt(zip_df).head(1000)

In [ ]:
regionname_df = zip_df.iloc[:, 2:]
regionname_df

In [ ]:
regionname_df.resample("Y")

# Melted Zip DF !!!!!

In [ ]:
zip_df = zip_df.reset_index()

In [ ]:
zip_df.drop(columns=['RegionID', 'SizeRank'], inplace=True)

In [ ]:
zip_df.head()

In [ ]:
 melted_zip_df = pd.melt(zip_df, id_vars=['RegionName'], var_name='time')

In [ ]:
melted_zip_df.info()

In [ ]:
melted_zip_df['time'] = pd.to_datetime(melted_zip_df['time'])

In [ ]:
melted_zip_df.info()

In [ ]:
melted_zip_df.rename(columns={"value": "home_sales"}, inplace=True)
melted_zip_df

In [ ]:
melted_zip_df = melted_zip_df.set_index('time')
melted_zip_df.head()

In [ ]:
melted_zip_df.resample('Y').mean()

In [ ]:
melted_zip_df.groupby('RegionName').mean()

## Test Melt 1

In [ ]:
test_melt = pd.melt(zip_1996, var_name='time')

In [ ]:
test_melt.info()

In [ ]:
test_melt['time'] = pd.to_datetime(melted['time'])

In [ ]:
test_melt.info()

## Test Melt 2

In [ ]:
test_melt_2 = pd.melt(regionname_df, var_name='time')

In [ ]:
test_melt_2

In [ ]:
test_melt_2.info()

In [ ]:
df.sort_values('RegionName')

## Grouping By State

In [ ]:
#recent_data.groupby('genres_list')['production_budget'].describe().sort_values('mean', ascending=False)
state_df = df.groupby('State')
state_df = state_df.mean() # Getting the mean value for each month
state_df.drop(columns=['RegionID', 'RegionName', 'SizeRank'], axis=1, inplace=True)
state_df.head()

In [ ]:
p2 = state_df.mean().plot(color='red')
p2

# Using Virginia as an example to figure out the process

In [ ]:
def melt_data(df):
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
va_df = state_df.iloc[45]
va_df = va_df.reset_index()
va_df.rename(columns={"index": "Date", "VA": "home_sales"}, inplace=True)
va_df['Date'] = pd.to_datetime(va_df['Date'])
va_df['home_sales'] = va_df['home_sales'].apply(lambda x: float("{:.2f}".format(x)))
va_df = va_df.set_index('Date')

In [ ]:
va_df.head()

## Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(50, 20))
p = sns.lineplot(data=va_df, x='Date', y='home_sales', color='blue');

p.set_xlabel("Date", fontsize = 50)
p.set_ylabel("Median Housing", fontsize = 50)

plt.xticks(fontsize=40)
plt.yticks(fontsize=40)

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick) 

p.set_title("Median Housing Sales in Virginia by Year", fontsize = 100)
plt.figsize=(50,25)

plt.show();

### Histogram (needs work)

In [ ]:
va_df.hist(figsize = (12,6))
plt.show()

### Density Plot

In [ ]:
va_df.plot(kind='kde', figsize = (12,6))
plt.show()

In [ ]:
# Determine rolling statistics
roll_mean = va_df.rolling(window=12, center=False).mean()
roll_std = va_df.rolling(window=12, center=False).std()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
p = sns.lineplot(data=va_df, x='Date', y='home_sales', color='blue', label='Original');
plt.plot(roll_mean, color='red', label='Rolling Mean')
plt.plot(roll_std, color='black', label = 'Rolling Std')

p.set_xlabel("Date", fontsize = 50)
p.set_ylabel("Median Housing", fontsize = 50)

plt.xticks(fontsize=40)
plt.yticks(fontsize=40)

fmt = '${x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)
ax.yaxis.set_major_formatter(tick) 

p.set_title("Median Housing Sales in Virginia by Year", fontsize = 100)
plt.figsize=(50,25)
plt.legend(loc='best')

plt.show();

## Analysis:
- Mean increases over time and therefore is not stationary

## Dickey Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform Dickey-Fuller test:
print ('Results of Dickey-Fuller Test: \n')
#dftest = adfuller(data['#Passengers'])
dftest = adfuller(va_df)

# Extract and display test results in a user friendly manner
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key,value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print(dfoutput)

# Data DF

# Turn Code from Virginia into a fuction

In [ ]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

In [ ]:
va_df = state_df.iloc[45]
va_df = va_df.reset_index()
va_df.rename(columns={"index": "Date", "VA": "home_sales"}, inplace=True)
va_df['Date'] = pd.to_datetime(va_df['Date'])
va_df['home_sales'] = va_df['home_sales'].apply(lambda x: float("{:.2f}".format(x)))
va_df = va_df.set_index('Date')

In [ ]:
state_df.head(2)

In [ ]:
state_df.iloc[:, 0]

In [ ]:
state_df.apply(lambda x:  )

In [ ]:
def get_state_data(df):
    
    for row in df:
        temp_df = .reset_index()
va_df.rename(columns={"index": "Date", "VA": "home_sales"}, inplace=True)
va_df['Date'] = pd.to_datetime(va_df['Date'])
va_df['home_sales'] = va_df['home_sales'].apply(lambda x: float("{:.2f}".format(x)))
va_df = va_df.set_index('Date')

# New Archive - 5/2

## Attempting to slice out a zip code and test

In [ ]:
zip_test = data_df.iloc[0]

In [ ]:
zip_test

In [ ]:
zip_test = zip_test.reset_index()

In [ ]:
zip_test.head()

In [ ]:
zip_test.rename(columns={"index": "date"}, inplace=True)

In [ ]:
zip_test['date'] = pd.to_datetime(zip_test['date'])

In [ ]:
zip_test.head()

In [ ]:
zip_test = zip_test.set_index('date')

In [ ]:
zip_test.resample("Y").mean()

In [ ]:
zip_test.info()

In [ ]:
zip_test.drop([0])

In [ ]:
df.iloc[:, 7:].mean()

In [ ]:
df.describe()

## Finding Average Values

## Melting State_df and attempting to get all the states

In [ ]:
state_df.head()

In [ ]:
 #melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'],
 #   var_name='time')

In [ ]:
state_df

In [ ]:
state_melt_df = pd.melt(state_df)

In [ ]:
state_melt_df

### Need to change columns to datetime objects

In [ ]:
state_df.columns

In [ ]:
state_df.columns = get_datetimes(state_df)

In [ ]:
#state_df = state_df.reset_index()

In [ ]:
state_df.head()

In [ ]:
state_df.groupby('State').sum().mean()

## Feature Engineering: Adding Total Mean Value & Annual Mean Value

In [ ]:
#state_df['1996'] =
#state_df.apply(lambda x: x[''] / x[''], axis=1)
df['Fruit Total']= df.iloc[:, -4:-1].sum(axis=1)

In [ ]:
state_df.iloc[:, 22:34].head(3)

In [ ]:
#sum_1996 = state_df.iloc[:, 1:10].sum(axis=1)
#sum_1997 = state_df.iloc[:, 10:22].sum(axis=1)
#sum_1998 = state_df.iloc[:, 22:34].sum(axis=1)
#sum_1999 = state_df.iloc[:, 34:46].sum(axis=1)
#sum_2000 = state_df.iloc[:, 46:58].sum(axis=1)
#sum_2001 = state_df.iloc[:, 58:70].sum(axis=1)
#sum_2002 = state_df.iloc[:, 70:82].sum(axis=1)
#sum_2003 = state_df.iloc[:, 82:94].sum(axis=1)
#sum_2004 = state_df.iloc[:, 94:106].sum(axis=1)
#sum_2005 = state_df.iloc[:, 106:118].sum(axis=1)
#sum_2006 = state_df.iloc[:, 118:130].sum(axis=1)

In [ ]:
#state_df['RegionID'] = state_df['RegionID'].astype(object)

In [ ]:
# plot feature importance
#fig, ax = plt.subplots(figsize=(50,20))
#p = sns.barplot(data=feature_importance_df, x='Importance', y='Feature', color ='mediumseagreen' );
#p.set_xlabel("Importance", fontsize = 50)

#p.set_ylabel("Feature", fontsize = 50)
#plt.xticks(fontsize=40)
#plt.yticks(fontsize=40)

#p.set_title("Features by Importance", fontsize = 100)
#plt.figsize=(30,20) 
#plt.savefig('images/project_3_Feature_Importance')

#plt.show();

In [ ]:
fig, ax = plt.subplots(figsize=(50, 50))
p = sns.lineplot(data=state_df);
p.set_xlabel("State", fontsize = 50)
p.set_ylabel("Median Home Value", fontsize = 50)

plt.xticks(fontsize=40)
plt.yticks(fontsize=40)

p.set_title("Title Goes Here", fontsize = 100)
plt.figsize=(50,25)

plt.show();

In [ ]:
state_df.mean().plot()

In [ ]:
df.reset_index()
df.head()

## Analysis of Categories

In [ ]:
#cat_df = df.iloc[:,0:7]
#cat_df.head()

In [ ]:
#cat_df.info()

In [ ]:
#cat_df['RegionName'] = cat_df['RegionName'].astype('object')
#cat_df.info()

In [ ]:
df.nunique()

In [ ]:
#print("RegionID unique values:", cat_df['RegionID'].nunique())
#print("RegionName unique values:", cat_df['RegionName'].nunique())
#print("City unique values:", cat_df['City'].nunique())
#print("State unique values:", cat_df['State'].nunique())
#print("Metro unique values:", cat_df['Metro'].nunique())
#print("CountyName unique values:", cat_df['CountyName'].nunique())
#print("SizeRank unique values:", cat_df['SizeRank'].nunique())

In [ ]:
#cat_df.sort_values(by = ['RegionName'], ascending = True).head()
df.sort_values(by = ['RegionName'], ascending = True).head()

A google search shows that RegionName is the ZipCode for each Region. However, upon sorting, I discovered that any ZipCode beginning with a 0 was ignoring it and displaying as a 4 digit number. I will use the .str().zfill() method to ensure that all RegionNames are displaying the as 5 digits.

In [ ]:
#cat_df['RegionName'] = cat_df['RegionName'].astype(str).str.zfill(5)
df['RegionName'] = df['RegionName'].astype(str).str.zfill(5)

In [ ]:
df.sort_values(by = ['RegionName'], ascending = True).head()

In [ ]:
#cat_df = cat_df.reset_index()
#cat_df

In [ ]:
#cat_df.sort_values(by = ['RegionName'], ascending = True).head()

That seems to have fixed it!

## Grouping Data

In [ ]:
metro_df = 

In [ ]:
melt_data(df)

## Analysis of Data Values

In [ ]:
#df['RegionID_copy'] = df['RegionID']

In [ ]:
#df.head()

In [ ]:
data_df = df.iloc[:, 7:]
data_df.head()

In [ ]:
data_df.columns = pd.to_datetime(data_df.columns, format='%Y-%m')
data_df.head()

In [ ]:
# changing values to thousands for ease of reading
data_df = data_df.applymap(lambda x: x/1000)
data_df.head()

In [ ]:
data_df.describe().round(2)

In [ ]:
#data_df.groupby(pd.Grouper(freq ='A'))

## Slicing out Years - data_df

In [ ]:
yr_1996 = data_df.iloc[:, :9]
yr_1996.head()

In [ ]:
yr_1997 = data_df.iloc[:, 9:21]
yr_1998 = data_df.iloc[:, 21:33]
yr_1999 = data_df.iloc[:, 33:45]

## CONTINUE TO SLICE OUT THE YEARS

In [ ]:
df.index

## EDA through Visualizations

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

plt.rc('font', **font)

In [ ]:
plt.gcf().autofmt_xdate()

In [ ]:
data_df.plot(figsize = (20,6), style = '.b');

## Melting Practice

In [ ]:
df.head(2)

In [ ]:
melt_test = df.iloc[:, :8]
melt_test.head()

In [ ]:
melt_data(melt_test)

In [ ]:
melted_df = melt_data(df)

In [ ]:
melted_df

## Merge Attempt

In [ ]:
#data_df = data_df.reset_index()

In [ ]:
#data_df

In [ ]:
#merged_df = cat_df.merge(data_df, on='index')

In [ ]:
#merged_df

## Slicing out Years and attempting to Melt - Full DF

In [ ]:
df.head()

In [ ]:
df.iloc[:, :16].head(2)

In [ ]:
yr_1996 = df.iloc[:, :16]
yr_1996.head()

In [ ]:
melt_data(yr_1996)

# Old Archive

In [ ]:
#data_df.isnull().sum()

In [ ]:
#df.columns[7:]

In [ ]:
#date_time_cols = pd.to_datetime(df.columns[7:])
#date_time_cols

In [ ]:
#cat_cols = df.columns[:7]
#cat_cols

In [ ]:
#new_cols = cat_cols + date_time_cols
#new_cols

In [ ]:
#df.rename(columns[7:] = date_time_cols)

## Exploring SizeRank

In [ ]:
df.sort_values('SizeRank').head()

In [ ]:
df.sort_values(by = ['State', 'City'], ascending = True)

In [ ]:
cat_data = df[['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank']]

In [ ]:
cat_data

In [ ]:
df.head()

In [ ]:
date_data = df.drop(['RegionID', 'RegionName', 'City', 'State', 'Metro', 'CountyName', 'SizeRank'], axis=1)

In [ ]:
date_data.head()

In [ ]:
date_time_data = get_datetimes(date_data)

In [ ]:
date_time_data

In [ ]:
len(date_time_data)

In [ ]:
list(date_data.columns)

In [ ]:
#df2 = date_data.append(date_time_data)

## Changing column names to DateTime Format

In [ ]:
#df.columns = pd.to_datetime(df.columns)
date_data.columns = pd.to_datetime(date_data.columns)

In [ ]:
date_data

### Merging with Categories

In [ ]:
#imdb_df = pd.merge(table7, table4, on= 'tconst', how='inner')
base_df = pd.merge(cat_data, date_data, how='outer')

# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

## Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

### Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='https://raw.githubusercontent.com/learn-co-students/dsc-mod-4-project-seattle-ds-102819/master/images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

## Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


## Step 1: Load the Data/Filtering for Chosen Zipcodes

## Step 2: Data Preprocessing

In [ ]:
def get_datetimes(df):
    """
    Takes a dataframe:
    returns only those column names that can be converted into datetime objects 
    as datetime objects.
    NOTE number of returned columns may not match total number of columns in passed dataframe
    """
    
    return pd.to_datetime(df.columns.values[1:], format='%Y-%m')

## Step 3: EDA and Visualization

In [ ]:
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 22}

matplotlib.rc('font', **font)

# NOTE: if you visualizations are too cluttered to read, try calling 'plt.gcf().autofmt_xdate()'!

## Step 4: Reshape from Wide to Long Format

In [ ]:
def melt_data(df):
    """
    Takes the zillow_data dataset in wide form or a subset of the zillow_dataset.  
    Returns a long-form datetime dataframe 
    with the datetime column names as the index and the values as the 'values' column.
    
    If more than one row is passes in the wide-form dataset, the values column
    will be the mean of the values from the datetime columns in all of the rows.
    """
    
    melted = pd.melt(df, id_vars=['RegionName', 'RegionID', 'SizeRank', 'City', 'State', 'Metro', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted.groupby('time').aggregate({'value':'mean'})

## Step 5: ARIMA Modeling

## Step 6: Interpreting Results